In [1]:
# pip install tensorflow
# pip install nltk

In [2]:
import tensorflow as tf
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
import warnings

warnings.simplefilter("ignore")
warnings.simplefilter("ignore", category=FutureWarning)

2022-12-14 12:09:07.049924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 12:09:07.178165: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 12:09:07.178188: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-14 12:09:07.843857: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
# stop_word
stop_word = set(stopwords.words('english'))
stop_word_symbol = {"…", "’", ":", '"', '-', '️', '&', '“', '(', '/', "'", ";", "+", "*", "~"}
stop_word.update(stop_word_symbol)

In [4]:
data = pd.read_csv("./TweetBLM.csv")
test_data = pd.read_csv("./crawl/tweets-new.csv")
data.drop_duplicates(subset=['tweet'], inplace=True)
test_data.drop_duplicates(subset=['tweet'], inplace=True)

In [5]:
# exporting tweet column
print('Number of Tweets: ', len(data['tweet'].tolist()))

data['tweet'] = data['tweet'].str.replace("RT (@[A-Za-z0-9_]+)|(@[A-Za-z0-9_]+)|https\S+|http\S+|(?<!\d)[.,;:!?](?!\d)", "")
test_data['tweet'] = test_data['tweet'].str.replace(
    "RT (@[A-Za-z0-9_]+)|(@[A-Za-z0-9_]+)|https\S+|http\S+|(?<!\d)[.,;:!?](?!\d)",
    "")

Number of Tweets:  9165


In [6]:
# Lemmatizer
tokenizer = TweetTokenizer(reduce_len=True)
lemmatizer = WordNetLemmatizer()

# Tokenize
data['tokenized'] = data['tweet'].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(x.lower()) if word not in stop_word])
test_data['tokenized'] = test_data['tweet'].apply(
    lambda x: [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(x.lower()) if word not in stop_word])

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# load the dataset
X_data, Y_data = data['tokenized'].values, data['label'].values
X_test, Y_test = test_data['tokenized'].values, test_data['label'].values

tk = Tokenizer()
tk.fit_on_texts(X_data)

total_cnt = len(tk.word_index)

tk = Tokenizer(total_cnt, oov_token='OOV')
tk.fit_on_texts(X_data)
X_data = tk.texts_to_sequences(X_data)
X_test = tk.texts_to_sequences(X_test)

max_len = 100
X_data = pad_sequences(X_data, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [8]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
# create the model

model = Sequential()
model.add(Embedding(total_cnt, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

# callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_GRU.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

2022-12-14 12:09:11.477369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-14 12:09:11.477404: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 12:09:11.477427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2022-12-14 12:09:11.477648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# train the model on the dataset
history = model.fit(X_data, Y_data, epochs=10, callbacks=[es, mc], batch_size=32, validation_split=0.2)

GRU_model = load_model('best_GRU.h5')
# evaluate the model on the test data
results = GRU_model.evaluate(X_test, Y_test)

print(results)

Epoch 1/10
229/230 [============================>.] - ETA: 0s - loss: 0.4358 - acc: 0.8311
Epoch 1: val_acc improved from -inf to 0.11566, saving model to best_GRU.h5
230/230 [==============================] - 17s 66ms/step - loss: 0.4357 - acc: 0.8312 - val_loss: 2.0475 - val_acc: 0.1157
Epoch 2/10
229/230 [============================>.] - ETA: 0s - loss: 0.2465 - acc: 0.8998
Epoch 2: val_acc improved from 0.11566 to 0.31151, saving model to best_GRU.h5
230/230 [==============================] - 15s 63ms/step - loss: 0.2463 - acc: 0.8999 - val_loss: 1.7566 - val_acc: 0.3115
Epoch 3/10
229/230 [============================>.] - ETA: 0s - loss: 0.1186 - acc: 0.9563
Epoch 3: val_acc improved from 0.31151 to 0.40862, saving model to best_GRU.h5
230/230 [==============================] - 15s 65ms/step - loss: 0.1187 - acc: 0.9562 - val_loss: 1.8855 - val_acc: 0.4086
Epoch 4/10
229/230 [============================>.] - ETA: 0s - loss: 0.0785 - acc: 0.9726
Epoch 4: val_acc did not improve 